In [1]:
from sqlalchemy import create_engine

import datetime
import numpy as np
import pandas as pd
import psycopg2
import os

In [2]:
SQL_SETTINGS = {
    'dbname'   : 'orderlogs', 
    'port'     :  5432, 
    'user'     : 'postgres', 
    'host'     : 'localhost',
    'password' : ''
}

ENGINE_SETTINGS = 'postgresql://postgres@localhost:5432/orderlogs'

with psycopg2.connect(**SQL_SETTINGS) as conn:
    cur = conn.cursor()
    cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public'")
    TABLE_NAMES = cur.fetchall()
    TABLE_NAMES = sorted(list(map(lambda x: x[0], TABLE_NAMES)))

TABLE_NAMES_ORDERLOGS = list(filter(lambda x: len(x) == 8, TABLE_NAMES))
TABLE_NAMES_CHAINS = list(filter(lambda x: len(x) > 8, TABLE_NAMES))

SECCODES = list(pd.read_csv(os.path.join(os.path.abspath(''), 
                                         'data_RTS_seccodes/seccodes.csv'), 
                            header=None).iloc[:,0])

In [3]:
def main():
    for table_name_orderlogs in TABLE_NAMES_ORDERLOGS:
        for name_instrument in SECCODES:
            dbwrite = DBWriteChains(date=table_name_orderlogs, 
                                    seccode=name_instrument)
            dbwrite.calculate()
            dbwrite.write()

In [4]:
class FilteredData:
    def __init__(self, *, date, seccode, buysell, seconds_gap):
        self._date = str(date) 
        self._seccode = str(seccode)
        self._buysell = str(buysell)
        self._seconds_gap = str(seconds_gap)
        
        self._query_filter = None
    
    def _query_filter_non_market_orders(self):
        '''This method creates the SQL query to generate
        filtered data (selected) from the DB by
        buysell, seccode, limit_order_flag = 1
        '''
        query_filter_non_market_orders = '''
        SELECT 
            *
        FROM 
            "{}"
        WHERE
            "BUYSELL" = '{}' AND
            "SECCODE" = '{}' AND
            ABS("PRICE") > 0.000001
        '''.format(self._date, self._buysell, self._seccode)
        query_filter_non_market_orders = ' '.join(
            query_filter_non_market_orders.split())
        
        return query_filter_non_market_orders
    
    def _query_out(self):
        '''This method creates the SQL query to generate further
        filtered data on the ordernos where exists only one row
        '''
        query_filter_non_market_orders = self._query_filter_non_market_orders()
        query_out = '''
        SELECT 
            limit_order_table."ORDERNO" as "ORDERNO", 
            array_agg(limit_order_table."NO") as "NOS", 
            array_agg(limit_order_table."TIME") as "TIMES", 
            array_agg(limit_order_table."ACTION") as "ACTIONS", 
            array_agg(limit_order_table."VOLUME") as "VOLUMES",
            array_agg(limit_order_table."PRICE") as "PRICES"
        FROM 
            ({}) limit_order_table
        GROUP BY
            "ORDERNO"
        HAVING
            array_length(array_agg(limit_order_table."ACTION"), 1) > 1
        ORDER BY
            "ORDERNO"
        '''.format(query_filter_non_market_orders)
        query_out = ' '.join(query_out.split())
        
        return query_out
    
    def _query_not_hft(self):
        '''This method generates the query to filter all hft orders by the 
        minutes_interval (all rows must occur during the specified interval)
        '''
        query_not_hft = '''
        SELECT 
            unnest(final."NOS") as "NO", 
            final."ORDERNO" as "ORDERNO", 
            final."MIN_TIME",
            final."MAX_TIME",
            unnest(final."TIMES") as "TIME", 
            unnest(final."ACTIONS") as "ACTION",
            unnest(final."VOLUMES") as "VOLUME",
            unnest(final."PRICES") as "PRICE"
        FROM
            (SELECT
                filtered."ORDERNO" as "ORDERNO",
                min(filtered."TIME") as "MIN_TIME",
                max(filtered."TIME") as "MAX_TIME",
                array_agg(filtered."NO") as "NOS",
                array_agg(filtered."TIME") as "TIMES",
                array_agg(filtered."ACTION") as "ACTIONS",
                array_agg(filtered."VOLUME") as "VOLUMES",
                array_agg(filtered."PRICE") as "PRICES"
            FROM
                (SELECT
                    out_table."ORDERNO" as "ORDERNO",
                    unnest(out_table."NOS") as "NO",
                    unnest(out_table."TIMES") as "TIME",
                    unnest(out_table."ACTIONS") as "ACTION",
                    unnest(out_table."VOLUMES") as "VOLUME",
                    unnest(out_table."PRICES") as "PRICE"
                FROM
                    ({}) out_table) filtered
            GROUP BY
                filtered."ORDERNO"
            HAVING
                (max(filtered."TIME") / 10000000000 * 60 * 60 * 1000000 + 
                 max(filtered."TIME") / 100000000 % 100 * 60 * 1000000 + 
                 max(filtered."TIME") / 1000000 % 100 * 1000000 - 
                 min(filtered."TIME") / 10000000000 * 60 * 60 * 1000000 -
                 min(filtered."TIME") / 100000000 % 100 * 60 * 1000000 - 
                 min(filtered."TIME") / 1000000 % 100 * 1000000) <= {} * 1000000) final
        GROUP BY
            final."ORDERNO",
            final."MIN_TIME",
            final."MAX_TIME",
            final."NOS",
            final."TIMES",
            final."ACTIONS",
            final."VOLUMES", 
            final."PRICES"
        ORDER BY
            "ORDERNO",
            "NO"
        '''.format(self._query_out(), self._seconds_gap)
        query_not_hft = ' '.join(query_not_hft.split())
        
        return query_not_hft
    
    def _query_not_iceberg(self):
        query_not_iceberg = '''
        SELECT
            unnest(not_hft_after_iceberg."NOS") as "NO",
            not_hft_after_iceberg."ORDERNO",
            unnest(not_hft_after_iceberg."TIMES") as "TIME",
            unnest(not_hft_after_iceberg."ACTIONS") as "ACTION",
            unnest(not_hft_after_iceberg."VOLUMES") as "VOLUME",
            unnest(not_hft_after_iceberg."PRICES") as "PRICE"
        FROM
            (SELECT
                array_agg(not_hft_before_iceberg."NO") as "NOS",
                not_hft_before_iceberg."ORDERNO", 
                array_agg(not_hft_before_iceberg."TIME") as "TIMES",
                array_agg(not_hft_before_iceberg."ACTION") as "ACTIONS",
                array_agg(not_hft_before_iceberg."VOLUME") as "VOLUMES",
                array_agg(not_hft_before_iceberg."PRICE") as "PRICES"
            FROM
                ({}) not_hft_before_iceberg
            GROUP BY
                not_hft_before_iceberg."ORDERNO"
            HAVING
                SUM(CASE
                    WHEN not_hft_before_iceberg."ACTION" = 1 THEN not_hft_before_iceberg."VOLUME"
                    ELSE -not_hft_before_iceberg."VOLUME"
                    END) >= 0) not_hft_after_iceberg
        ORDER BY
            "ORDERNO"
        '''.format(self._query_not_hft())
        query_not_iceberg = ' '.join(query_not_iceberg.split())

        return query_not_iceberg
    
    def _query_for_calculation(self):
        query_filtered = self._query_not_iceberg()

        result_query = '''
        SELECT 
            dum.*
        FROM
            (SELECT 
                (array_agg("NO"))[1] as "FIRST_NO",
                non_hft."ORDERNO",
                MAX(non_hft."VOLUME") as "VOLUME",
                MIN(non_hft."TIME") as "MIN_TIME",
                MAX(non_hft."TIME") as "MAX_TIME",
                1 as "FLAG",
                -1 as "CHAIN_NUMBER"
            FROM
                ({}) non_hft
            GROUP BY
                "ORDERNO"
            HAVING
                not (array_agg(non_hft."ACTION")::int[] && array[0])
            UNION ALL
            SELECT 
                (array_agg("NO"))[1] as "FIRST_NO",
                non_hft."ORDERNO",
                MAX(non_hft."VOLUME") as "VOLUME",
                MIN(non_hft."TIME") as "MIN_TIME",
                MAX(non_hft."TIME") as "MAX_TIME",
                0 as "FLAG",
                -1 as "CHAIN_NUMBER"
            FROM
                ({}) non_hft
            GROUP BY
                "ORDERNO"
            HAVING
                (array_agg(non_hft."ACTION")::int[] && array[0])) dum
        ORDER BY
            "VOLUME", 
            "FIRST_NO"
        '''.format(query_filtered, query_filtered)
        result_query = ' '.join(result_query.split())
        
        return result_query
    
    def query_generate(self):
        '''This method generates the query according to the specified 
        hidden methods for creation the query of data filter
        '''
        self._query_filter = self._query_for_calculation()
        
        return self._query_filter
    
    def db_query_run(self, connection_parameters):
        '''This method runs the query with the connection to database
        
        :connection_parameters: connection parameters for the connect 
            object of psycopg2 (dict)
        '''
        with psycopg2.connect(**connection_parameters) as conn:
            cur = conn.cursor()
            cur.execute(self._query_filter)
            result = cur.fetchall()
        
        return pd.DataFrame(result)

In [5]:
def time_convertion(time: int):
    if len(str(time)) == 12:
        time = str(time)
        microseconds = int(time[-6:])
        seconds = int(time[-8:-6])
        minutes = int(time[-10:-8])
        hours = int(time[-12:-10])

        return (hours * 60**2 + minutes * 60 + seconds) * 10**6 + microseconds
    else:
        return 0

In [6]:
class ChainIdentification:
    def __init__(self, *, date, seccode, buysell, seconds_gap=1):
        self._date = date
        self._seccode = seccode
        self._buysell = buysell
        self._seconds_gap = seconds_gap 
        
    def _chain_identification(self):
        data = FilteredData(date=self._date, seccode=self._seccode, 
                            buysell=self._buysell, seconds_gap=self._seconds_gap)
        data.query_generate()
        df = data.db_query_run(SQL_SETTINGS)
        
        microseconds_gap = self._seconds_gap * 10**6

        identified_orderno = set()
        chains = {}
        
        indicator = np.round(np.arange(0, len(df)) / len(df) * 100, 2)

        for row in range(len(df)):
            if row % 10000 == 0:
                print(str(indicator[row]) + '%', self._buysell)
            if not (df.iloc[row, 1] in identified_orderno):
                if df.iloc[row, 2] in chains.keys():
                    chain_current = chains[df.iloc[row, 2]]
                    list_times_chains = np.array(list(map(lambda x: time_convertion(x), 
                                                          map(lambda x: x[-1], 
                                                              chain_current))))
                    index = np.logical_and((time_convertion(df.iloc[row, 3]) - list_times_chains) < microseconds_gap,
                                           (time_convertion(df.iloc[row, 3]) - list_times_chains) >= 0)
                    index = np.argwhere(index == True)
                    if any(index):
                        index = time_convertion(df.iloc[row, 3]) - list_times_chains
                        index = np.argmin(index[index >= 0])
                        if df.iloc[row, 5] == 0:
                            chains[df.iloc[row, 2]][index].pop()
                            chains[df.iloc[row, 2]][index].extend([df.iloc[row, 1], df.iloc[row, 4]])
                            identified_orderno.add(df.iloc[row, 1])
                        else:
                            chains[df.iloc[row, 2]][index].pop()
                            chains[df.iloc[row, 2]][index].extend([df.iloc[row, 1], 0])
                            identified_orderno.add(df.iloc[row, 1])
                    else:
                        if df.iloc[row, 5] == 0:
                            chains[df.iloc[row, 2]].append([df.iloc[row, 1], df.iloc[row, 4]])
                            identified_orderno.add(df.iloc[row, 1])
                else:
                    if df.iloc[row, 5] == 0:
                        chains[df.iloc[row, 2]] = [[df.iloc[row, 1], df.iloc[row, 4]]]
                identified_orderno.add(df.iloc[row, 1])
            
        return chains 
    
    def chain_calculation(self):
        chains = self._chain_identification()
        dictlist = []
        for key, value in chains.items():
            temp = list(map(lambda x: x[:-1], value))
            dictlist.append(temp)

        flatten = [item for sublist in dictlist for item in sublist]
        flatten = list(zip(range(1, len(flatten) + 1), flatten))

        chains_result = []
        for row in flatten:
            for item in row[1]:
                chains_result.append([item, row[0]])
        chains_result = pd.DataFrame(chains_result, 
                                     columns=['ORDERNO', 
                                              'CHAIN_NUMBER']).sort_values(by='ORDERNO').reset_index(drop=True)
        
        return chains_result

In [7]:
class DBWriteChains:
    def __init__(self, *, date, seccode, seconds_gap=1):
        self._date = date
        self._seccode = seccode
        self._seconds_gap = seconds_gap
        
        self._df = None
    
    def calculate(self):
        result_buy = ChainIdentification(date=self._date, seccode=self._seccode,
                                         seconds_gap=self._seconds_gap,
                                         buysell='B').chain_calculation()
        result_sell = ChainIdentification(date=self._date, seccode=self._seccode,
                                          seconds_gap=self._seconds_gap,
                                          buysell='S').chain_calculation()
        result = pd.concat((result_buy, result_sell)).sort_values(by='ORDERNO').reset_index(drop=True)
        
        self._df = result
        
        return self._df
    
    def write(self):
        if self._df is not None:
            table_name = str(self._date) + '_' + str(self._seccode)
            engine = create_engine(ENGINE_SETTINGS)
            self._df.to_sql(table_name, con=engine)
        else:
            print('You must calculate chains at once')

In [8]:
if __name__ == '__main__':
    main()

0.0% B
3.26% B
6.52% B
9.79% B
13.05% B
16.31% B
19.57% B
22.84% B
26.1% B
29.36% B
32.62% B
35.88% B
39.15% B
42.41% B
45.67% B
48.93% B
52.2% B
55.46% B
58.72% B
61.98% B
65.24% B
68.51% B
71.77% B
75.03% B
78.29% B
81.55% B
84.82% B
88.08% B
91.34% B
94.6% B
97.87% B
0.0% S
2.74% S
5.48% S
8.22% S
10.96% S
13.7% S
16.44% S
19.19% S
21.93% S
24.67% S
27.41% S
30.15% S
32.89% S
35.63% S
38.37% S
41.11% S
43.85% S
46.59% S
49.33% S
52.08% S
54.82% S
57.56% S
60.3% S
63.04% S
65.78% S
68.52% S
71.26% S
74.0% S
76.74% S
79.48% S
82.22% S
84.97% S
87.71% S
90.45% S
93.19% S
95.93% S
98.67% S
0.0% B
5.75% B
11.49% B
17.24% B
22.98% B
28.73% B
34.47% B
40.22% B
45.96% B
51.71% B
57.45% B
63.2% B
68.94% B
74.69% B
80.43% B
86.18% B
91.92% B
97.67% B
0.0% S
5.91% S
11.83% S
17.74% S
23.66% S
29.57% S
35.48% S
41.4% S
47.31% S
53.22% S
59.14% S
65.05% S
70.97% S
76.88% S
82.79% S
88.71% S
94.62% S
0.0% B
1.09% B
2.18% B
3.27% B
4.36% B
5.45% B
6.54% B
7.63% B
8.72% B
9.81% B
10.9% B
11.99% B
1

48.78% B
53.66% B
58.53% B
63.41% B
68.29% B
73.17% B
78.05% B
82.92% B
87.8% B
92.68% B
97.56% B
0.0% S
4.79% S
9.57% S
14.36% S
19.15% S
23.93% S
28.72% S
33.51% S
38.29% S
43.08% S
47.87% S
52.65% S
57.44% S
62.23% S
67.01% S
71.8% S
76.59% S
81.38% S
86.16% S
90.95% S
95.74% S
0.0% B
10.99% B
21.98% B
32.97% B
43.96% B
54.95% B
65.94% B
76.93% B
87.92% B
98.91% B
0.0% S
10.71% S
21.41% S
32.12% S
42.82% S
53.53% S
64.24% S
74.94% S
85.65% S
96.35% S
0.0% B
6.3% B
12.61% B
18.91% B
25.21% B
31.51% B
37.82% B
44.12% B
50.42% B
56.73% B
63.03% B
69.33% B
75.64% B
81.94% B
88.24% B
94.54% B
0.0% S
5.81% S
11.61% S
17.42% S
23.22% S
29.03% S
34.84% S
40.64% S
46.45% S
52.25% S
58.06% S
63.87% S
69.67% S
75.48% S
81.28% S
87.09% S
92.9% S
98.7% S
0.0% B
19.4% B
38.8% B
58.2% B
77.61% B
97.01% B
0.0% S
14.22% S
28.44% S
42.67% S
56.89% S
71.11% S
85.33% S
99.56% S
0.0% B
61.8% B
0.0% S
50.9% S
0.0% B
30.22% B
60.45% B
90.67% B
0.0% S
21.1% S
42.19% S
63.29% S
84.39% S
0.0% B
40.68% B
81.3

67.49% B
80.99% B
94.49% B
0.0% S
12.97% S
25.95% S
38.92% S
51.89% S
64.86% S
77.84% S
90.81% S
0.0% B
7.64% B
15.27% B
22.91% B
30.54% B
38.18% B
45.81% B
53.45% B
61.09% B
68.72% B
76.36% B
83.99% B
91.63% B
99.26% B
0.0% S
7.41% S
14.82% S
22.22% S
29.63% S
37.04% S
44.45% S
51.85% S
59.26% S
66.67% S
74.08% S
81.48% S
88.89% S
96.3% S
0.0% B
23.82% B
47.63% B
71.45% B
95.26% B
0.0% S
16.86% S
33.71% S
50.57% S
67.43% S
84.28% S
0.0% B
59.14% B
0.0% S
64.11% S
0.0% B
25.97% B
51.95% B
77.92% B
0.0% S
21.11% S
42.21% S
63.32% S
84.43% S
0.0% B
34.5% B
69.01% B
0.0% S
26.96% S
53.93% S
80.89% S
0.0% B
90.91% B
0.0% S
0.0% B
50.58% B
0.0% S
46.99% S
93.98% S
0.0% B
28.04% B
56.07% B
84.11% B
0.0% S
40.57% S
81.14% S
0.0% B
53.32% B
0.0% S
41.31% S
82.62% S
0.0% B
32.47% B
64.95% B
97.42% B
0.0% S
41.01% S
82.03% S
0.0% B
68.88% B
0.0% S
79.31% S
0.0% B
25.87% B
51.74% B
77.61% B
0.0% S
26.9% S
53.79% S
80.69% S
0.0% B
26.44% B
52.89% B
79.33% B
0.0% S
55.66% S
0.0% B
78.41% B
0.0% S
6

49.87% S
54.86% S
59.85% S
64.83% S
69.82% S
74.81% S
79.8% S
84.78% S
89.77% S
94.76% S
99.74% S
0.0% B
17.44% B
34.89% B
52.33% B
69.78% B
87.22% B
0.0% S
18.29% S
36.57% S
54.86% S
73.15% S
91.44% S
0.0% B
12.26% B
24.52% B
36.79% B
49.05% B
61.31% B
73.57% B
85.83% B
98.1% B
0.0% S
8.38% S
16.77% S
25.15% S
33.53% S
41.91% S
50.3% S
58.68% S
67.06% S
75.45% S
83.83% S
92.21% S
0.0% B
21.64% B
43.27% B
64.91% B
86.54% B
0.0% S
16.44% S
32.88% S
49.32% S
65.76% S
82.2% S
98.64% S
0.0% B
50.68% B
0.0% S
49.75% S
99.5% S
0.0% B
31.88% B
63.76% B
95.64% B
0.0% S
39.87% S
79.74% S
0.0% B
43.94% B
87.89% B
0.0% S
47.32% S
94.64% S
0.0% B
0.0% S
0.0% B
33.31% B
66.63% B
99.94% B
0.0% S
38.17% S
76.34% S
0.0% B
53.32% B
0.0% S
60.01% S
0.0% B
42.23% B
84.46% B
0.0% S
41.12% S
82.24% S
0.0% B
22.47% B
44.95% B
67.42% B
89.9% B
0.0% S
27.24% S
54.48% S
81.71% S
0.0% B
69.11% B
0.0% S
92.13% S
0.0% B
52.97% B
0.0% S
77.68% S
0.0% B
31.88% B
63.77% B
95.65% B
0.0% S
29.22% S
58.44% S
87.67% S
0

24.15% S
48.3% S
72.45% S
96.6% S
0.0% B
19.18% B
38.37% B
57.55% B
76.74% B
95.92% B
0.0% S
17.82% S
35.65% S
53.47% S
71.3% S
89.12% S
0.0% B
42.71% B
85.42% B
0.0% S
29.99% S
59.99% S
89.98% S
0.0% B
0.0% S
70.33% S
0.0% B
58.89% B
0.0% S
54.12% S
0.0% B
56.47% B
0.0% S
47.49% S
94.98% S
0.0% B
0.0% S
0.0% B
58.28% B
0.0% S
54.84% S
0.0% B
58.16% B
0.0% S
49.99% S
99.98% S
0.0% B
86.41% B
0.0% S
72.31% S
0.0% B
0.0% S
0.0% B
0.0% S
0.0% B
0.0% S
0.0% B
66.46% B
0.0% S
58.06% S
0.0% B
88.51% B
0.0% S
99.11% S
0.0% B
66.61% B
0.0% S
50.14% S
0.0% B
61.85% B
0.0% S
61.02% S
0.0% B
28.59% B
57.18% B
85.76% B
0.0% S
37.82% S
75.64% S
0.0% B
24.94% B
49.88% B
74.81% B
99.75% B
0.0% S
32.23% S
64.46% S
96.69% S
0.0% B
28.44% B
56.89% B
85.33% B
0.0% S
25.34% S
50.68% S
76.03% S
0.0% B
49.94% B
99.89% B
0.0% S
46.74% S
93.47% S
0.0% B
44.98% B
89.96% B
0.0% S
50.14% S
0.0% B
0.0% S
0.0% B
0.0% S
0.0% B
0.0% S
0.0% B
0.0% S
0.0% B
0.0% S
0.0% B
0.0% S
0.0% B
94.8% B
0.0% S
0.0% B
0.0% S
0.0%